# **ESG CLEANING CODE**

*Please be careful when running cells* - the toggled cells run as one so be EXTRA careful with this as we don't want to re-run the final cell in each toggle (this sends the table to BQ and will overwrite what we already have there). This isn't an issue right now but might be in the future!!

In [ ]:
from google.colab import auth

In [ ]:
from google.colab import auth
import pandas as pd

# Will collect your credentials
auth.authenticate_user()


In [ ]:
import pandas_gbq

In [ ]:
#@title (ESG_ONLY table)

# NOTE: this is a test only using the ESG_ONLY table:

# STEP 1: Query BigQuery to get the Before Ticker table for ESG data:
query1 = "SELECT * FROM `lw-esg-project.Enriched_SP500_fin_london.fin_adj_close`"

fin_melt = pd.read_gbq(query1, project_id = 'lw-esg-project')

fin_melt.head()

<ipython-input-26-9b1ccf6959d2>:8: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  fin_melt = pd.read_gbq(query1, project_id = 'lw-esg-project')


,the_date,date_date,AAPL,ABBV,ABT,ACN,ADBE,ADI,ADM,ADP,...,UNM,MLM,FLS,AIV,PCG,KSS,UAA,HOG,JWN,HP
0,2024-08-02,2024-08-02 04:00:00+00:00,219.364471,186.036041,110.209518,321.839966,526.169983,206.271927,57.865154,260.843262,...,53.244980,566.662476,46.465313,8.316000,18.377859,19.054470,6.50,35.442234,20.777672,35.683659
1,2023-10-16,2023-10-16 04:00:00+00:00,177.614731,140.710861,89.904655,300.509186,550.739990,169.071762,71.481491,242.435471,...,48.664062,428.660004,37.996670,6.141333,16.352798,16.923464,7.00,29.563906,13.444651,40.862556
2,2023-07-05,2023-07-05 04:00:00+00:00,189.890305,128.795090,103.646118,302.638947,487.260010,185.074753,73.869812,212.620850,...,45.628368,445.536316,35.669304,7.989334,17.757883,20.949724,7.25,34.884617,18.857304,33.034218
3,2024-01-18,2024-01-18 05:00:00+00:00,187.710403,157.560440,111.742767,355.153687,593.700012,186.801163,66.345924,231.008270,...,44.294292,500.304260,39.498146,7.000000,16.830545,22.222391,7.50,33.186443,17.059267,31.672432
4,2022-09-22,2022-09-22 04:00:00+00:00,150.900192,130.047318,96.188042,253.007111,287.059998,139.623306,80.848373,219.443756,...,37.213116,323.157562,24.300962,7.541333,12.914824,21.955618,7.75,37.080696,16.741352,36.123302


In [ ]:
fin_melt.drop(columns=['date_date'],inplace=True)

In [ ]:
# STEP 2: Renaming columns to just tickers:
new_columns = []
for col in fin_melt.columns:
  col.split('Ticker')[-1]
  new_columns.append(col.split('Ticker')[-1])
fin_melt.columns = new_columns

fin_melt.head()

,the_date,AAPL,ABBV,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,...,UNM,MLM,FLS,AIV,PCG,KSS,UAA,HOG,JWN,HP
0,2024-08-02,219.364471,186.036041,110.209518,321.839966,526.169983,206.271927,57.865154,260.843262,231.369995,...,53.244980,566.662476,46.465313,8.316000,18.377859,19.054470,6.50,35.442234,20.777672,35.683659
1,2023-10-16,177.614731,140.710861,89.904655,300.509186,550.739990,169.071762,71.481491,242.435471,211.779999,...,48.664062,428.660004,37.996670,6.141333,16.352798,16.923464,7.00,29.563906,13.444651,40.862556
2,2023-07-05,189.890305,128.795090,103.646118,302.638947,487.260010,185.074753,73.869812,212.620850,203.850006,...,45.628368,445.536316,35.669304,7.989334,17.757883,20.949724,7.25,34.884617,18.857304,33.034218
3,2024-01-18,187.710403,157.560440,111.742767,355.153687,593.700012,186.801163,66.345924,231.008270,241.179993,...,44.294292,500.304260,39.498146,7.000000,16.830545,22.222391,7.50,33.186443,17.059267,31.672432
4,2022-09-22,150.900192,130.047318,96.188042,253.007111,287.059998,139.623306,80.848373,219.443756,187.149994,...,37.213116,323.157562,24.300962,7.541333,12.914824,21.955618,7.75,37.080696,16.741352,36.123302


In [ ]:
# STEP 3: Melt esg_before table to make a table that Looker can work nicely with:

melt_fin = pd.melt(frame = fin_melt, id_vars = 'the_date', var_name = 'Ticker', value_name = 'Adjusted_Close')

melt_fin.head()

# NOTE: This makes a melted table from ONE of the split tables

,the_date,Ticker,Adjusted_Close
0,2024-08-02,AAPL,219.364471
1,2023-10-16,AAPL,177.614731
2,2023-07-05,AAPL,189.890305
3,2024-01-18,AAPL,187.710403
4,2022-09-22,AAPL,150.900192


In [ ]:
melt_fin = melt_fin.sort_values(by=['Ticker', 'the_date'], ascending=True)
melt_fin

,the_date,Ticker,Adjusted_Close
1329,2014-01-02,AAPL,17.234297
430,2014-01-03,AAPL,16.855730
742,2014-01-06,AAPL,16.947649
1757,2014-01-07,AAPL,16.826447
2109,2014-01-08,AAPL,16.932999
...,...,...,...
822935,2025-01-24,ZTS,168.610001
824364,2025-01-27,ZTS,173.029999
822974,2025-01-28,ZTS,170.419998
822641,2025-01-29,ZTS,170.220001


In [ ]:
# STEP 8: Sending this melted table back to BQ: THIS IS ALL OF THE BEFORE TABLES MELTED:
pandas_gbq.to_gbq(melt_fin, 'Enriched_SP500_fin_london.melted_fin_adj_close', project_id = 'lw-esg-project', if_exists = 'replace')

100%|██████████| 1/1 [00:00<00:00, 1560.96it/s]
